In [48]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

import numpy as np
from scipy.special import factorial2

hbarc = 197.3 #MeV fm
hbar = 6.582E-22 #MeV s
alpha = 1/137 
mP = 938.272 # Mev/c^2
#muN = 3.152E-14 # MeV / T
muN2 =  alpha*hbarc**3*2*mP**-2 # nuclear magneton squared

def Thalf_width(E): #converts energy width (MeV) to Thalf in seconds
    return hbar * np.log(2)/E

def Thalf_width_kev(E):
    if E==0.:
        return None
    return Thalf_width(E/1000)

def times_so_far(ls):
    out = [0]*len(ls)
    for i in range(len(ls)):
        out[i] = ls[:i+1].count(ls[i])
    return out

def j_to_twoj(x):
    if '/2' in x:
        return int(x.strip('/2'))
    else:
        return 2*int(x.strip('()'))
    
def thalf_units_to_sec(units,value):
    m=0
    if units=='PS':
        m = 10**-12
    elif units=='NS':
        m = 10**-9
    elif units=='FS':
        m = 10**-15
    elif units=='US':
        m = 10**-6
    elif units=='AS':
        m = 10**-18
    elif units=='KEV':
        pass
    if m==0:
        return Thalf_width_kev(value)
    else:
        return m*value
    
def my_float(x):
    try:
        return float(x)
    except:
        return 0.0
    
def fLambda(l):
    return 8*np.pi*(l+1)/(l*(factorial2(2*l+1)**2))

# def BEl(l,Egamma,Thalf,I,units):
#     if units == 1: # [B] = fm^2l
#         B = (I/100)*(np.log(2)/Thalf)*fLambda(l)**-1*hbar*hbarc**(2*l+1)*(Egamma**(-2*l-1))
#     elif units == 2: # [B] = e^2 fm^2l
#         B = (I/100)*(np.log(2)/Thalf)*fLambda(l)**-1*hbar*hbarc**(2*l+1)*(Egamma**(-2*l-1))
#         B = B/(alpha*hbarc)
#     elif units == 3: # [B] = e^2 b^l
#         B = (I/100)*(np.log(2)/Thalf)*fLambda(l)**-1*hbar*hbarc**(2*l+1)*(Egamma**(-2*l-1))
#         B = 10**(-2*l)*B/(alpha*hbarc)
#     return B

def BM1(Egamma,Thalf):
    return (np.log(2)/Thalf) * (1/1.76E13) * Egamma**-3 

# def BE2W(A,Egamma,Thalf,I):
#     lamW = 7.3E7*A**(4/3)*Egamma**5;
#     lamExp = (np.log(2)/Thalf)*(I/100);
#     return lamExp/lamW

# def Bweisskopf(l,A):
#     # Weisskopf (single-particle) estimate in e^2 fm^2l
#     return (1/(4*np.pi)) * (3/(3+l))**2 * (1.2*A**(1/3))**(2*l)

# def BEl_in_Wu(l,Egamma,Thalf,I,A):
#     return BEl(l,Egamma,Thalf,I,2) / Bweisskopf(l,A)

# def BWu_convert(BEl_Wu,l,A):
#     return BEl_Wu*Bweisskopf(l,A)

BM1_Weisskopf = (10/np.pi)*0.75**2


In [49]:
# df = pd.read_csv('sd_E2_data.csv',sep=r'\s*,\s*',header=0, encoding='ascii', engine='python')
df = pd.read_csv('M1_ensdf.csv')
df = df.rename(columns=lambda x: x.strip())

In [50]:
df

,A,Element,Z,N,Energy (keV),Energy Unc (keV),Ji,Thalf,E_gamma (keV),E_gamma Unc (keV),M Gamma,B_exp (W.u.),B_exp Unc (W.u.)
0,16,O \t,8,8,9844.5000,5,2+ \t,0.62 KEV 10 \t,2927.1000,8 \t,[M1] \t,4.200000e-03,\t8
1,18,O \t,8,10,3920.4400,14,2+ \t,18.4 FS 20 \t,1938.0000,\t,M1 \t,1.400000e-01,\t2
2,18,O \t,8,10,5254.8000,9,2+ \t,7.0 FS 3 \t,1334.0000,\t,M1 \t,1.110000e-01,\t8
3,18,O \t,8,10,7116.9000,12,4+ \t,17 FS LT \t,3562.0000,\t,M1 \t,7.100000e-02,\t16
4,18,O \t,8,10,8213.0000,4,2+ \t,1.0 KEV 8 \t,4293.0000,\t,M1 \t,7.200000e-03,\t30
5,19,O \t,8,11,96.0000,5,3/2+ \t,1.39 NS 5 \t,96.0000,\t,M1 \t,8.800000e-02,\t38
6,19,O \t,8,11,1471.7000,4,1/2+ \t,0.88 PS 12 \t,1375.7000,\t,M1 \t,9.300000e-03,\t15
7,18,F \t,9,9,1041.5500,8,0+ \t,1.77 FS 31 \t,1041.0000,\t,M1 \t,1.090000e+01,\t21
8,18,F \t,9,9,1700.8100,18,1+ \t,662 FS 19 \t,659.0000,\t,M1 \t,8.050000e-02,\t27
9,18,F \t,9,9,1700.8100,18,1+ \t,662 FS 19 \t,1701.0000,\t,M1 \t,2.000000e-03,\t11


In [51]:
df[df['A']==22]

,A,Element,Z,N,Energy (keV),Energy Unc (keV),Ji,Thalf,E_gamma (keV),E_gamma Unc (keV),M Gamma,B_exp (W.u.),B_exp Unc (W.u.)
83,22,F \t,9,13,709.10,3,(2+) \t,57 FS 28 \t,637.40,2 \t,[M1] \t,1.500000,\t8
100,22,NE \t,10,12,4456.20,9,2+ \t,3.7 FS 25 \t,3181.40,\t,M1+E2 \t,0.180000,\t12
101,22,NE \t,10,12,5329.60,13,1+ \t,1.2 FS 3 \t,4054.60,\t,M1+E2 \t,0.020000,\t11
102,22,NE \t,10,12,5363.40,11,2+ \t,69 FS 12 \t,4088.40,\t,M1+E2 \t,0.003900,\t8
103,22,NE \t,10,12,5523.30,6,(4)+ \t,21 FS 3 \t,2166.10,5 \t,M1 \t,0.102000,\t15
104,22,NE \t,10,12,5641.20,7,3+ \t,3 FS LT \t,4366.10,10 \t,M1+E2 \t,0.059000,\t>
105,22,NE \t,10,12,6119.90,16,2+ \t,14 FS 7 \t,4844.80,\t,M1+E2 \t,0.001700,\t10
106,22,NE \t,10,12,6119.90,16,2+ \t,14 FS 7 \t,1663.60,\t,M1+E2 \t,0.012000,\t8
107,22,NE \t,10,12,6345.10,10,4+ \t,13 FS 3 \t,2987.70,9 \t,M1+E2 \t,0.043000,\t12
108,22,NE \t,10,12,6819.40,16,2+ \t,3 FS LT \t,2363.10,\t,M1+E2 \t,0.021000,\t>


In [52]:
def isNaN(num):
    return num != num

for i in df.index:
    if isNaN(df['A'].loc[i]):
#         print(df['A'].loc[i])
        df = df.drop(i)
    
df['A'] = df['A'].map(int)
df['Z'] = df['Z'].map(int)
df['N'] = df['N'].map(int)

In [53]:
# df['Element'] = df['Element'].apply(lambda x: str.strip(x))

for key in df:
    try:
        df[key] = df[key].apply(lambda x: str.strip(x))
    except:
        pass


df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df['Element'] = df['Element'].apply(str.capitalize)

In [54]:
df = df.rename(columns={'B(W.u.)':'B_exp (W.u.)'})
df = df.rename(columns={'B Unc (W.u.)':'B_exp Unc (W.u.)'})

In [55]:
# df['Jpi'] = df['Jpi'].map(lambda x: str(x).strip().strip('()+'))
# df = df.rename(columns={'Jpi':'Ji'})
df['Ji'] = df['Ji'].map(lambda x: str(x).strip().strip('()+'))
# df = df.rename(columns={'Jpi':'Ji'})

df['twoJi'] = df['Ji'].map(j_to_twoj)

In [56]:
df['Include'] = True

In [57]:
for i in df.index:
    string = df['B_exp Unc (W.u.)'].loc[i]
    try:
        up,down = string.strip('+').split('-')
        df['B_exp Unc (W.u.)'].loc[i] = max(float(up),float(down))
    except:
        pass
    if string=='':
        df['B_exp Unc (W.u.)'].loc[i] = df['B_exp (W.u.)'].loc[i]/10
    elif (string=='GT') | (string=='>'):
        df['Include'].loc[i] = False
    elif string=='<':
        df['B_exp Unc (W.u.)'].loc[i] = df['B_exp (W.u.)'].loc[i]
      

/tmp/ipykernel_3108/177498008.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Include'].loc[i] = False
/tmp/ipykernel_3108/177498008.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['B_exp Unc (W.u.)'].loc[i] = df['B_exp (W.u.)'].loc[i]/10
/tmp/ipykernel_3108/177498008.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['B_exp Unc (W.u.)'].loc[i] = max(float(up),float(down))
/tmp/ipykernel_3108/177498008.py:13: Settin

In [58]:
for i in df.index:
    A = df['A'].loc[i]
#     Z = df['Z'].loc[i]
#     N = df['N'].loc[i]
    if (A<18) | (A>38):
        df['Include'].loc[i] = False

/tmp/ipykernel_3108/164583894.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Include'].loc[i] = False


In [59]:
for i in df.index:
    E = df['Energy (keV)'].loc[i]
    if E>8000:
        df['Include'].loc[i] = False

/tmp/ipykernel_3108/3972538932.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Include'].loc[i] = False


In [60]:
df

,A,Element,Z,N,Energy (keV),Energy Unc (keV),Ji,Thalf,E_gamma (keV),E_gamma Unc (keV),M Gamma,B_exp (W.u.),B_exp Unc (W.u.),twoJi,Include
0,16,O,8,8,9844.5000,5,2,0.62 KEV 10,2927.1000,8,[M1],4.200000e-03,8,4,False
1,18,O,8,10,3920.4400,14,2,18.4 FS 20,1938.0000,,M1,1.400000e-01,2,4,True
2,18,O,8,10,5254.8000,9,2,7.0 FS 3,1334.0000,,M1,1.110000e-01,8,4,True
3,18,O,8,10,7116.9000,12,4,17 FS LT,3562.0000,,M1,7.100000e-02,16,8,True
4,18,O,8,10,8213.0000,4,2,1.0 KEV 8,4293.0000,,M1,7.200000e-03,30,4,False
5,19,O,8,11,96.0000,5,3/2,1.39 NS 5,96.0000,,M1,8.800000e-02,38,3,True
6,19,O,8,11,1471.7000,4,1/2,0.88 PS 12,1375.7000,,M1,9.300000e-03,15,1,True
7,18,F,9,9,1041.5500,8,0,1.77 FS 31,1041.0000,,M1,1.090000e+01,21,0,True
8,18,F,9,9,1700.8100,18,1,662 FS 19,659.0000,,M1,8.050000e-02,27,2,True
9,18,F,9,9,1700.8100,18,1,662 FS 19,1701.0000,,M1,2.000000e-03,11,2,True


In [61]:
# df[['Thalf Value','Thalf Units','Thalf Unc']] = df['Thalf'].str.split(expand=True)

# df = df[df['Thalf Unc']!='LT']
# df = df[df['Thalf Unc']!='LE']
# df = df[df['Thalf Unc']!='GT']
# # df = df[df['I_gamma Unc']!='AP']

# df['Thalf Unc'] = df['Thalf Unc'].fillna(0.0)
# df['E_gamma Unc (keV)'] = df['E_gamma Unc (keV)'].fillna(0.0)
# # df['I_gamma Unc'] = df['I_gamma Unc'].fillna(0.0)

# for i in df.index:
#     string = df['Thalf Unc'].loc[i]
#     try:
#         up,down = string.strip('+').split('-')
#         df['Thalf Unc'].loc[i] = max(float(up),float(down))
#     except:
#         pass

# df['Thalf (sec)'] = df[['Thalf Units','Thalf Value']].apply(lambda row: thalf_units_to_sec(row['Thalf Units'],float(row['Thalf Value'])),axis=1,result_type='expand')
# df['Thalf Unc (sec)'] = df[['Thalf Units','Thalf Unc']].apply(lambda row: thalf_units_to_sec(row['Thalf Units'],float(row['Thalf Unc'])),axis=1,result_type='expand')

In [62]:
# for col in ['I_gamma','I_gamma Unc','E_gamma (keV)','E_gamma Unc (keV)']:
#     df[col] = df[col].apply(my_float)

# df['B_exp (e^2fm^4)'] = df[['E_gamma (keV)','Thalf (sec)','I_gamma']].apply(lambda row: BEl(2,row['E_gamma (keV)']/1000,row['Thalf (sec)'],row['I_gamma'],2),axis=1,result_type='expand')

# NOTE: the 5 on the E_gamma term below comes from the derivative dB/dE and is 5 = |-2*lambda - 1|
# df['B_exp Unc (e^2fm^4)'] = df.apply(lambda row: row['B_exp (e^2fm^4)']*np.sqrt( (row['I_gamma Unc']/row['I_gamma'])**2 + (5 * row['E_gamma Unc (keV)']/row['E_gamma (keV)'])**2 + (row['Thalf Unc (sec)']/row['Thalf (sec)'])**2 ) , axis=1 ,result_type='expand' ) 

In [63]:
# df['B_exp (mu_N^2)'] = -9.
# df['B_exp Unc (mu_N^2)'] = -9.

# df['B_exp Unc (W.u.)'] = df['B_exp Unc (W.u.)'][df['B_exp Unc (W.u.)'] != '<']
# df['B_exp Unc (W.u.)'] = df['B_exp Unc (W.u.)'][df['B_exp Unc (W.u.)'] != '>']

# for i in df.index:
# #     print(df['B_exp (W.u.)'].loc[i])
# #     print(df['B_exp Unc (W.u.)'].loc[i])
# #     print(df['A'].loc[i])
#     df['B_exp (mu_N^2)'].loc[i] = BM1_Weisskopf * float(df['B_exp (W.u.)'].loc[i])
#     df['B_exp Unc (mu_N^2)'].loc[i] = BM1_Weisskopf * float(df['B_exp Unc (W.u.)'].loc[i])
    

In [64]:
df['Nucleus'] = df.apply(lambda x: str(int(x['A']))+x['Element'], axis=1)

In [65]:
df_levels = pd.read_csv('sd_levels_20mev.csv')

df_levels['E(keV)'] = None
df_levels['dE(keV)'] = None
    
for i in df_levels.index:
#     print(df_levels['Elevel(keV)'].iloc[i])
    E = df_levels['Elevel(keV)'].loc[i].split()
    if len(E)==1:
        df_levels['E(keV)'].loc[i] = float(E[0].replace('?',''))
        df_levels['dE(keV)'].loc[i] = None
    else:
        df_levels['E(keV)'].loc[i] = float(E[0].replace('?',''))
        df_levels['dE(keV)'].loc[i] = float(E[1].replace('?',''))  

# df_levels = df_levels.drop('unclean_E',axis=1)
df_levels = df_levels.drop('T1/2',axis=1)
# df_levels = df_levels.drop('unclean_Jpi',axis=1)
# df_levels = df_levels.drop('unclean_T1/2',axis=1)
df_levels['Include'] = True
for char in [',','&','-','GE',':','TO']:
    df_levels['Include'] *= df_levels['Jpi'].map(lambda x : not char in str(x))
df_levels = df_levels.drop(df_levels[df_levels['Include']==False].index)
for char in ['+','(',')','[',']']:
    df_levels['Jpi'] = df_levels['Jpi'].map(lambda x: str(x).replace(char,''))
df_levels = df_levels.rename(columns={'Jpi':'J'})
df_levels = df_levels.drop(df_levels[df_levels['J']==''].index)
df_levels['twoJ'] = df_levels['J'].map(j_to_twoj)

df_levels['Element'] = df_levels['Nucleus'].map(lambda x : x[2:])
df_levels['Element'] = df_levels['Element'].apply(str.capitalize)
df_levels['A'] = df_levels['Nucleus'].map(lambda x : x[:2])
df_levels['Nucleus'] = df_levels.apply(lambda x: str(int(x['A']))+x['Element'], axis=1)

In [66]:
df_levels['Nucleus'].unique()

array(['11O', '12O', '14O', '15O', '16O', '17O', '18O', '19O', '20O',
       '21O', '22O', '23O', '24O', '26O', '15F', '16F', '17F', '18F',
       '19F', '20F', '21F', '22F', '23F', '24F', '25F', '26F', '27F',
       '16Ne', '18Ne', '19Ne', '20Ne', '21Ne', '22Ne', '23Ne', '24Ne',
       '25Ne', '26Ne', '27Ne', '28Ne', '29Ne', '30Ne', '32Ne', '34Ne',
       '17Na', '19Na', '20Na', '21Na', '22Na', '23Na', '24Na', '25Na',
       '26Na', '27Na', '28Na', '29Na', '30Na', '31Na', '33Na', '35Na',
       '20Mg', '21Mg', '22Mg', '23Mg', '24Mg', '25Mg', '26Mg', '27Mg',
       '28Mg', '29Mg', '30Mg', '31Mg', '32Mg', '34Mg', '36Mg', '38Mg',
       '40Mg', '21Al', '22Al', '23Al', '24Al', '25Al', '26Al', '27Al',
       '28Al', '29Al', '30Al', '31Al', '32Al', '33Al', '35Al', '39Al',
       '22Si', '23Si', '24Si', '25Si', '26Si', '27Si', '28Si', '29Si',
       '30Si', '31Si', '32Si', '33Si', '34Si', '35Si', '36Si', '37Si',
       '38Si', '39Si', '40Si', '42Si', '44Si', '25P', '26P', '27P', '28P',
     

In [67]:
df_levels['n'] = None
for nucleus in df_levels['Nucleus'].unique():
    ls = list(df_levels[df_levels['Nucleus']==nucleus]['twoJ'])
#     print(ls)
    tsf = times_so_far(ls)
    j = 0
    for i in df_levels[df_levels['Nucleus']==nucleus].index:
#         print(i)
        df_levels['n'].loc[i] = tsf[j]
        j+=1

/tmp/ipykernel_3108/602153835.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_levels['n'].loc[i] = tsf[j]


In [68]:
df_levels.head()

,Nucleus,Elevel(keV),J,E(keV),dE(keV),Include,twoJ,Element,A,n
0,11O,490,5/2,490.0,None,True,5,O,11,1
1,11O,2120,5/2,2120.0,None,True,5,O,11,2
2,12O,0,0,0.0,None,True,0,O,12,1
3,12O,1.62E3 11,0,1620.0,11.0,True,0,O,12,2
4,12O,1968 52,2,1968.0,52.0,True,4,O,12,1


In [69]:
df['Jf'] = None
df['twoJf'] = None
df['ni'] = None
df['nf'] = None

max_deltaJ = 1
tol_energy = 50
tol_gamma = 50

# for i in df.index:
for i in df.index:
    nuc_cond = (df_levels['Nucleus']==df['Nucleus'].loc[i])
    i_J_cond = (df_levels['twoJ']==df['twoJi'].loc[i])
    i_E_cond = (tol_energy > abs(df_levels['E(keV)'] - df['Energy (keV)'].loc[i]))
    f_J_cond = ( 2*max_deltaJ >= abs(df_levels['twoJ'] - df['twoJi'].loc[i]))
    f_E_cond = (tol_gamma > abs(df_levels['E(keV)'] - df['Energy (keV)'].loc[i] + df['E_gamma (keV)'].loc[i]))
    i_condition =  nuc_cond &  i_J_cond  &  i_E_cond
    f_condition =  nuc_cond &  f_J_cond  &  f_E_cond
    state_i = df_levels[i_condition]
    state_f = df_levels[f_condition]
    
#     print('len=',len(state_i))
    
    if (len(state_i)!=1)|(len(state_f)!=1):
        df['Include'].loc[i] = False
        print(f"-------------------------------------------")
        print(f"Excluding transition i = {i}")
        print(f'Matched # initial states: {len(state_i)}' )
        print(f'Matched # final   states: {len(state_f)}' )
        print(f"Matched # Ji             : { len( df_levels[ nuc_cond & i_J_cond ] ) }")
        print(f"Matched # Jf             : { len( df_levels[ nuc_cond & f_J_cond ] ) }")
        print(f"Matched # Ei             : { len( df_levels[ nuc_cond & i_E_cond ] ) }")
        print(f"Matched # Ef             : { len( df_levels[ nuc_cond & f_E_cond ] ) }")
        print(state_i)
        print(state_f)
        print(df.loc[i])

        continue
        
    df['Jf'].loc[i] = list(state_f['J'])[0]
    df['twoJf'].loc[i] = int(state_f['twoJ'])
    df['ni'].loc[i] = int(state_i['n'])
    df['nf'].loc[i] = int(state_f['n'])
    
#     print(state_i)
#     print(state_f)
#     print(df.loc[i])
    

/tmp/ipykernel_3108/2465717445.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Jf'].loc[i] = list(state_f['J'])[0]
/tmp/ipykernel_3108/2465717445.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['twoJf'].loc[i] = int(state_f['twoJ'])
/tmp/ipykernel_3108/2465717445.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ni'].loc[i] = int(state_i['n'])
/tmp/ipykernel_3108/2465717445.py:43: SettingWithCopyWarning: 
A valu

-------------------------------------------
Excluding transition i = 98
Matched # initial states: 2
Matched # final   states: 1
Matched # Ji             : 5
Matched # Jf             : 14
Matched # Ei             : 3
Matched # Ef             : 1
    Nucleus Elevel(keV)    J  E(keV) dE(keV)  Include  twoJ Element   A  n
685    21Ne   6267.0 16  9/2  6267.0    16.0     True     9      Ne  21  2
686    21Ne    6271.3 7  9/2  6271.3     7.0     True     9      Ne  21  3
    Nucleus Elevel(keV)    J  E(keV) dE(keV)  Include  twoJ Element   A  n
674    21Ne    2866.6 2  9/2  2866.6     2.0     True     9      Ne  21  1
A                          21
Element                    Ne
Z                          10
N                          11
Energy (keV)           6267.0
Energy Unc (keV)           16
Ji                        9/2
Thalf                24 FS 12
E_gamma (keV)          3400.1
E_gamma Unc (keV)          16
M Gamma                 M1+E2
B_exp (W.u.)            0.016
B_exp Unc (W.u.)    

/tmp/ipykernel_3108/2465717445.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Include'].loc[i] = False
/tmp/ipykernel_3108/2465717445.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Jf'].loc[i] = list(state_f['J'])[0]
/tmp/ipykernel_3108/2465717445.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['twoJf'].loc[i] = int(state_f['twoJ'])
/tmp/ipykernel_3108/2465717445.py:42: SettingWithCopyWarning: 
A value is tr

-------------------------------------------
Excluding transition i = 191
Matched # initial states: 1
Matched # final   states: 2
Matched # Ji             : 14
Matched # Jf             : 31
Matched # Ei             : 1
Matched # Ef             : 3
     Nucleus Elevel(keV)  J   E(keV) dE(keV)  Include  twoJ Element   A  n
1644    26Mg   5476.05 7  4  5476.05     7.0     True     8      Mg  26  3
     Nucleus Elevel(keV)  J   E(keV) dE(keV)  Include  twoJ Element   A  n
1637    26Mg   4318.89 5  4  4318.89     5.0     True     8      Mg  26  1
1639    26Mg   4350.09 4  3  4350.09     4.0     True     6      Mg  26  2
A                         26
Element                   Mg
Z                         12
N                         14
Energy (keV)         5476.05
Energy Unc (keV)           7
Ji                         4
Thalf                21 FS 6
E_gamma (keV)        1157.23
E_gamma Unc (keV)          6
M Gamma                M1+E2
B_exp (W.u.)            0.34
B_exp Unc (W.u.)          10
t

/tmp/ipykernel_3108/2465717445.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Include'].loc[i] = False
/tmp/ipykernel_3108/2465717445.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Include'].loc[i] = False
/tmp/ipykernel_3108/2465717445.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Include'].loc[i] = False
/tmp/ipykernel_3108/2465717445.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy

...

-------------------------------------------
Excluding transition i = 309
Matched # initial states: 1
Matched # final   states: 2
Matched # Ji             : 24
Matched # Jf             : 67
Matched # Ei             : 2
Matched # Ef             : 3
     Nucleus Elevel(keV)  J   E(keV) dE(keV)  Include  twoJ Element   A   n
2016    26Al   5849.21 8  2  5849.21     8.0     True     4      Al  26  11
     Nucleus Elevel(keV)  J   E(keV) dE(keV)  Include  twoJ Element   A  n
1975    26Al   2069.47 3  2  2069.47     3.0     True     4      Al  26  2
1976    26Al   2071.64 4  1  2071.64     4.0     True     2      Al  26  3
A                         26
Element                   Al
Z                         13
N                         13
Energy (keV)         5849.21
Energy Unc (keV)           8
Ji                         2
Thalf                10 FS 6
E_gamma (keV)        3779.44
E_gamma Unc (keV)           
M Gamma                 [M1]
B_exp (W.u.)            0.02
B_exp Unc (W.u.)          12

/tmp/ipykernel_3108/2465717445.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Include'].loc[i] = False
/tmp/ipykernel_3108/2465717445.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Include'].loc[i] = False
/tmp/ipykernel_3108/2465717445.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Jf'].loc[i] = list(state_f['J'])[0]
/tmp/ipykernel_3108/2465717445.py:41: SettingWithCopyWarning: 
A value is trying to be se

-------------------------------------------
Excluding transition i = 403
Matched # initial states: 1
Matched # final   states: 2
Matched # Ji             : 5
Matched # Jf             : 6
Matched # Ei             : 1
Matched # Ef             : 2
     Nucleus Elevel(keV)  J  E(keV) dE(keV)  Include  twoJ Element   A  n
2686    30Si    5372.2 6  0  5372.2     6.0     True     0      Si  30  3
     Nucleus Elevel(keV)  J   E(keV) dE(keV)  Include  twoJ Element   A  n
2680    30Si   3769.48 4  1  3769.48     4.0     True     2      Si  30  1
2681    30Si   3787.72 4  0  3787.72     4.0     True     0      Si  30  2
A                          30
Element                    Si
Z                          14
N                          16
Energy (keV)           5372.2
Energy Unc (keV)            6
Ji                          0
Thalf                59 FS 21
E_gamma (keV)          1602.8
E_gamma Unc (keV)           9
M Gamma                    M1
B_exp (W.u.)            0.036
B_exp Unc (W.u.)      

/tmp/ipykernel_3108/2465717445.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Include'].loc[i] = False
/tmp/ipykernel_3108/2465717445.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Jf'].loc[i] = list(state_f['J'])[0]
/tmp/ipykernel_3108/2465717445.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['twoJf'].loc[i] = int(state_f['twoJ'])
/tmp/ipykernel_3108/2465717445.py:42: SettingWithCopyWarning: 
A value is tr

-------------------------------------------
Excluding transition i = 497
Matched # initial states: 1
Matched # final   states: 2
Matched # Ji             : 4
Matched # Jf             : 5
Matched # Ei             : 1
Matched # Ef             : 3
     Nucleus Elevel(keV)  J   E(keV) dE(keV)  Include  twoJ Element   A  n
3834     34S  6251.22 19  4  6251.22    19.0     True     8       S  34  3
     Nucleus  Elevel(keV)  J    E(keV) dE(keV)  Include  twoJ Element   A  n
3826     34S  4876.839 24  3  4876.839    24.0     True     6       S  34  1
3827     34S      4882 14  4    4882.0    14.0     True     8       S  34  2
A                                34
Element                           S
Z                                16
N                                18
Energy (keV)                6251.22
Energy Unc (keV)                 19
Ji                                4
Thalf                0.42 PS +49-21
E_gamma (keV)               1374.34
E_gamma Unc (keV)                20
M Gamma       

In [70]:
# df_levels['Nucleus']==df['Nucleus'].loc[i]

In [71]:
df[df['Include']==False]

,A,Element,Z,N,Energy (keV),Energy Unc (keV),Ji,Thalf,E_gamma (keV),E_gamma Unc (keV),M Gamma,B_exp (W.u.),B_exp Unc (W.u.),twoJi,Include,Nucleus,Jf,twoJf,ni,nf
0,16,O,8,8,9844.5000,5,2,0.62 KEV 10,2927.1000,8,[M1],4.200000e-03,8,4,False,16O,2,4,2,1
4,18,O,8,10,8213.0000,4,2,1.0 KEV 8,4293.0000,,M1,7.200000e-03,30,4,False,18O,2,4,4,2
10,18,F,9,9,3061.8400,18,2,0.83 FS LT,3061.0000,,M1,2.000000e-01,GT,4,False,18F,1,2,2,1
11,18,F,9,9,3061.8400,18,2,0.83 FS LT,2125.0000,,M1,2.000000e+00,GT,4,False,18F,3,6,2,1
63,19,F,9,10,8137.7000,12,1/2,0.3 KEV LE,1883.0000,,M1,2.800000e-01,0.028,1,False,19F,1/2,1,6,4
64,19,F,9,10,8137.7000,12,1/2,0.3 KEV LE,4229.5000,,M1,4.400000e-01,0.044,1,False,19F,3/2,3,6,2
65,19,F,9,10,8137.7000,12,1/2,0.3 KEV LE,2200.0000,,M1,5.800000e-01,0.058,1,False,19F,1/2,1,6,3
66,19,F,9,10,8310.0000,12,5/2,0.047 KEV 19,3932.0000,,M1,2.200000e-01,5,5,False,19F,7/2,7,10,1
90,21,Ne,10,11,3735.5900,14,5/2,10 FS LT,3735.2000,2,M1+E2,3.400000e-02,>,5,False,21Ne,3/2,3,2,1
91,21,Ne,10,11,3735.5900,14,5/2,10 FS LT,3384.6000,2,M1+E2,4.600000e-03,>,5,False,21Ne,5/2,5,2,1


In [72]:
df[(df['Include']==True)]

,A,Element,Z,N,Energy (keV),Energy Unc (keV),Ji,Thalf,E_gamma (keV),E_gamma Unc (keV),M Gamma,B_exp (W.u.),B_exp Unc (W.u.),twoJi,Include,Nucleus,Jf,twoJf,ni,nf
1,18,O,8,10,3920.4400,14,2,18.4 FS 20,1938.0000,,M1,0.140000,2,4,True,18O,2,4,2,1
2,18,O,8,10,5254.8000,9,2,7.0 FS 3,1334.0000,,M1,0.111000,8,4,True,18O,2,4,3,2
3,18,O,8,10,7116.9000,12,4,17 FS LT,3562.0000,,M1,0.071000,16,8,True,18O,4,8,2,1
5,19,O,8,11,96.0000,5,3/2,1.39 NS 5,96.0000,,M1,0.088000,38,3,True,19O,5/2,5,1,1
6,19,O,8,11,1471.7000,4,1/2,0.88 PS 12,1375.7000,,M1,0.009300,15,1,True,19O,3/2,3,1,1
7,18,F,9,9,1041.5500,8,0,1.77 FS 31,1041.0000,,M1,10.900000,21,0,True,18F,1,2,1,1
8,18,F,9,9,1700.8100,18,1,662 FS 19,659.0000,,M1,0.080500,27,2,True,18F,0,0,2,1
9,18,F,9,9,1700.8100,18,1,662 FS 19,1701.0000,,M1,0.002000,11,2,True,18F,1,2,2,1
12,18,F,9,9,3358.2000,10,3,0.30 PS 2,835.0000,,M1,0.007400,37,6,True,18F,2,4,2,1
13,18,F,9,9,3358.2000,10,3,0.30 PS 2,2421.0000,,M1,0.000470,17,6,True,18F,3,6,2,1


In [52]:
# df_output = df[df['Include']==True]

In [53]:
df_output.to_csv('cleaned_sd_M1_data_v3.csv',index=False)